In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [68]:
data = pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv').head(800000).sort_values(['city'])
data.head()

/var/folders/0x/j2sfsgp167vf2w83dyjt6_nm0000gn/T/ipykernel_12174/2033632590.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv').head(800000).sort_values(['city'])


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
147552,190034,2021/02/12,LA,190034,ABBEVILLE GENERAL HOSPITAL,118 N HOSPITAL DR,ABBEVILLE,70510.0,Short Term,22113.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
577721,421301,2022/09/02,SC,421301,ABBEVILLE AREA MEDICAL CENTER,420 THOMSON CIRCLE,ABBEVILLE,29620.0,Critical Access Hospitals,45001.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
70132,421301,2022/12/30,SC,421301,ABBEVILLE AREA MEDICAL CENTER,420 THOMSON CIRCLE,ABBEVILLE,29620.0,Critical Access Hospitals,45001.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
571322,190034,2020/12/04,LA,190034,ABBEVILLE GENERAL HOSPITAL,118 N HOSPITAL DR,ABBEVILLE,70510.0,Short Term,22113.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
149854,190034,2023/01/27,LA,190034,ABBEVILLE GENERAL HOSPITAL,118 N HOSPITAL DR,ABBEVILLE,70510.0,Short Term,22113.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0


In [69]:
len(data)

742253

In [70]:
def ifZero(x):
    if x < 0:
        return 0
    return x

In [71]:
clean = data
for col in clean.columns:
  if clean[col].dtype != 'float64':
    clean = clean.drop(col, axis=1)
clean = clean.drop(['zip', 'fips_code'], axis=1)
clean

,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,inpatient_beds_used_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,...,previous_day_admission_pediatric_covid_confirmed_0_4_7_day_sum,previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum,previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_sum
147552,42.4,40.4,40.4,18.7,18.7,-999999.0,-999999.0,-999999.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577721,NaN,NaN,18.0,11.4,11.4,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70132,NaN,NaN,18.0,13.0,13.0,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571322,36.0,34.0,34.0,20.7,20.7,-999999.0,-999999.0,-999999.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149854,NaN,NaN,56.0,27.9,27.9,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739122,101.0,NaN,NaN,19.1,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739631,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
740480,10.0,10.0,10.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741230,NaN,NaN,10.0,-999999.0,-999999.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
clean = clean.dropna()
clean = clean.applymap(lambda x: ifZero(x))
clean

,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,inpatient_beds_used_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,...,previous_day_admission_pediatric_covid_confirmed_0_4_7_day_sum,previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum,previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_sum
520722,38.3,38.3,38.1,23.0,23.0,9.7,9.7,9.7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
723582,35.0,35.0,25.0,9.4,9.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
527290,33.6,33.6,25.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381698,35.0,35.0,25.0,9.1,9.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
283564,35.0,35.0,25.0,16.3,16.3,5.4,5.4,4.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161385,205.9,196.0,202.1,205.9,202.1,19.3,17.4,11.3,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504377,96.5,96.5,97.1,72.4,72.4,17.3,17.3,17.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
653196,16.0,16.0,16.0,13.6,13.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
716067,186.0,186.0,169.0,147.6,147.6,26.9,26.9,25.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
output = clean['inpatient_beds_used_covid_7_day_sum'].values.reshape(-1, 1)
clean = clean.drop('inpatient_beds_used_covid_7_day_sum', axis=1)
'inpatient_beds_used_covid_7_day_sum' in clean

False

In [74]:
solve = []
maxTrainAccuracy = -1
maxTestAccuracy = -1
index = -1
for col in clean:
  input = clean[col].values.reshape(-1, 1)
  xtrain, xtest, ytrain, ytest = train_test_split(input, output, test_size=0.3, random_state=404)
  dtree = DecisionTreeClassifier(random_state=404)
  dtree.fit(xtrain, ytrain)
  y_pred = dtree.predict(xtest)
  trainAccuracy = accuracy_score(y_true = ytrain, y_pred = dtree.predict(xtrain))
  testAccuracy = accuracy_score(y_true = ytest, y_pred = y_pred)
  
  if testAccuracy > maxTestAccuracy:
    maxTrainAccuracy = trainAccuracy
    maxTestAccuracy = testAccuracy
    index = len(solve)
    
  solve.append({
    'col': col,
    'input': input,
    'output': output,
    'xtrain': xtrain,
    'xtest': xtest,
    'ytrain': ytrain,
    'ytest': ytest,
    'dtree': dtree,
    'y_pred': y_pred,
    'trainAccuracy': trainAccuracy,
    'testAccuracy': testAccuracy,
  })
  

In [75]:
print('name:',solve[index].get('col'))
print("index:",index)
print('max training accuracy',maxTrainAccuracy)
print('max test accuracy',maxTestAccuracy)

name: total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum
index: 25
max training accuracy 0.8710464310464311
max test accuracy 0.8624369421808304
